In [ ]:
!pip install nltk

# Обработка

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

nltk.download('punkt_tab')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
stopwords = []
with open('english.csv', 'r', encoding='utf-8') as f:
  stopwords += [word.strip() for word in f.readlines()]

In [3]:
def preprocess(text):
    text_filtered = [word.lower() for word in word_tokenize(text, language='english') if word.isalpha()]
    words_filtered = [stemmer.stem(word) for word in text_filtered if word not in stopwords]

    return ' '.join(words_filtered)

In [4]:
df = pd.read_csv("sample.tsv", sep="\t",  names=['tag', 'text'], on_bad_lines='warn')

ham_text = ''
spam_text = ''

for index, row in df.iterrows():
    row['text'] = preprocess(row['text'])
    if row['tag'] == 'ham':
        ham_text += row['text'] + ' '
    else:
        spam_text += row['text'] + ' '

<ipython-input-4-8b15803823c2>:1: ParserWarning: Skipping line 802: expected 2 fields, saw 3

  df = pd.read_csv("sample.tsv", sep="\t",  names=['tag', 'text'], on_bad_lines='warn')


# Задание для базы

In [5]:
ham_alph = [
      i for i in ham_text.split()
      if len(i) >= 3
      ]

ham_alph = sorted(set(ham_alph), key=lambda x: ham_alph.count(x), reverse=True)

spam_alph = [
      i for i in spam_text.split()
      if len(i) >= 3
      ]

spam_alph = sorted(set(spam_alph), key=lambda x: spam_alph.count(x), reverse=True)

In [6]:
print(ham_alph[:20], spam_alph[:20], sep='\n')

['get', 'come', 'call', 'like', 'know', 'got', 'want', 'good', 'love', 'time', 'day', 'one', 'sorri', 'need', 'home', 'take', 'think', 'see', 'send', 'work']
['call', 'free', 'text', 'txt', 'mobil', 'claim', 'stop', 'repli', 'week', 'prize', 'new', 'send', 'get', 'servic', 'award', 'tone', 'nokia', 'min', 'urgent', 'chat']


# Задание для хардкора

In [7]:
full_words = ham_text + ' ' + spam_text
full_words_list = full_words.split()

words_alph = []
for i in full_words_list:
  if full_words_list.count(i) < 10:
    words_alph.append(i)
  elif len(i) < 3:
    words_alph.append(i)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

In [9]:
vectorizer = CountVectorizer(stop_words=words_alph)

matrix = vectorizer.fit_transform(df.text)
df1 = pd.DataFrame(data= matrix.toarray(), columns = vectorizer.get_feature_names_out())
normalize(df1, norm="l2")
df1['tag'] = df['tag']

df1

,abiola,abl,abt,account,activ,actual,address,aft,afternoon,age,...,wrong,xmas,xxx,yeah,year,yes,yesterday,yet,yup,tag
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,spam
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,spam
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
3710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
3711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
3712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
